In [1]:
import pandas as pd
import numpy as np

results = pd.read_csv('results.csv')
results.head()

,date,year,home_team,away_team,home_score,away_score,total_goals,win_margin,tournament,city,country,neutral
0,2000-01-04,2000,Egypt,Togo,2,1,3,1,Friendly,Aswan,Egypt,False
1,2000-01-07,2000,Tunisia,Togo,7,0,7,7,Friendly,Tunis,Tunisia,False
2,2000-01-08,2000,Trinidad and Tobago,Canada,0,0,0,0,Friendly,Port of Spain,Trinidad and Tobago,False
3,2000-01-09,2000,Burkina Faso,Gabon,1,1,2,0,Friendly,Ouagadougou,Burkina Faso,False
4,2000-01-09,2000,Guatemala,Armenia,1,1,2,0,Friendly,Los Angeles,United States,True


#### 1. Count the Number of Unique Home Teams and Away Teams

In [2]:
unique_num_home = results['home_team'].nunique()
unique_num_away = results['home_team'].nunique()

unique_num_home, unique_num_away

(245, 245)

#### 2. Tournament Statistics

For each tournament
- Compute the number of games played
- Sum up the total number of goals 
- Obtain the maximum and minimum win margins

In [3]:
tournament_stats = results.groupby('tournament').agg({
    'date': 'count',
    'total_goals': 'sum',
    'win_margin': ['max', 'min']
})

tournament_stats

date total_goals win_margin    
                                     count         sum        max min
tournament                                                           
AFC Asian Cup qualification            465        1480         20  -8
African Cup of Nations                 421         933          5  -5
African Cup of Nations qualification  1012        2388          7 -10
CECAFA Cup                             341         858          9  -9
FIFA World Cup                         384         965          8  -6
FIFA World Cup qualification          4942       14174         31 -12
Friendly                              7472       18853         15  -9
Gold Cup                               297         796          7  -6
UEFA Euro qualification               1293        3593         11 -13
UEFA Nations League                    468        1139          6  -6

#### 3. Analyzing the 2022 FIFA World Cup

Calculate the Total Number of Goals by Each Team in the FIFA World Cup 2022. 

Hint: We'll need to combine each team's goals scored as both the home team and away team. 

1. **Split** `results` into a DataFrame containing only FIFA World Cup 2022 matches
- Combine multiple boolean masks using the `&` operator:
    - `results['tournament']=='FIFA World Cup'`
    - `results['year']== 2022` 

- Perform a second split that creates separate DataFrames for home teams and away teams

2. **Apply** aggregation functions to each DataFrame that sums up the total number of goals scored by each team
- Don't forget to flatten the index of the resulting aggregated DataFrames

3. **Combine** the two DataFrames by performing a **left** join where the DataFrame on the left contains the data for the home team (important!)
- Feel free to clean the column names **but don't drop the `home_team` and `away_team` columns!**
- Create a new column `total_goals` that adds up the total goals scored by each team as the home team and away team
- Sort from highest to lowest number of `total_goals` by each team.

In [4]:
# SPLIT

# 1. Create a filter mask
fwc_2022_matches = (results['tournament'] == 'FIFA World Cup') & (results['year'] == 2022)

# 2. Apply the mask
fwc_2022_df = results[fwc_2022_matches]

# 3. Display the new DataFrame
fwc_2022_df.head()

,date,year,home_team,away_team,home_score,away_score,total_goals,win_margin,tournament,city,country,neutral
17021,2022-11-20,2022,Qatar,Ecuador,0,2,2,-2,FIFA World Cup,Al Khor,Qatar,False
17022,2022-11-21,2022,Senegal,Netherlands,0,2,2,-2,FIFA World Cup,Doha,Qatar,True
17023,2022-11-21,2022,England,Iran,6,2,8,4,FIFA World Cup,Al Rayyan,Qatar,True
17024,2022-11-21,2022,United States,Wales,1,1,2,0,FIFA World Cup,Al Rayyan,Qatar,True
17025,2022-11-22,2022,Argentina,Saudi Arabia,1,2,3,-1,FIFA World Cup,Lusail,Qatar,True


In [5]:
# APPLY

# 1. Use filter mask and create split DataFrame for home and away teams
# - For home_teams
home_goals = fwc_2022_df.groupby('home_team')['home_score'].sum().reset_index(name='home_goals')
home_goals.head()

,home_team,home_goals
0,Argentina,11
1,Australia,1
2,Belgium,1
3,Brazil,7
4,Cameroon,4


In [6]:
# - For away_teams
away_goals = fwc_2022_df.groupby('away_team')['away_score'].sum().reset_index(name='away_goals')
away_goals.head()

,away_team,away_goals
0,Argentina,4
1,Australia,3
2,Belgium,0
3,Brazil,1
4,Cameroon,0


In [7]:
# COMBINE

# 1. Rename 'away_team' and 'away_goals' in away_goals DataFrame to match home_goals DataFrame
away_goals.rename(columns={
    'away_team': 'team', 'away_goals': 'away_goals'}, inplace=True)
home_goals.rename(columns={
    'home_team': 'team', 'home_goals': 'home_goals'}, inplace=True)

# 2. Perform a `left` join
combined_goals = pd.merge(home_goals, away_goals, on='team', how='left')

# 3. Fill `NaN` with 0 for teams that didn't score away goals
combined_goals['away_goals'].fillna(0, inplace=True)

# 4. Create new column for 'total_goals'
combined_goals['total_goals'] = combined_goals['home_goals']  + combined_goals['away_goals']

# 5. Sort DataFrame by 'total_goals' in descending order
combined_goals = combined_goals.sort_values(by='total_goals', ascending=False)

#  6. Display the new DataFrame
combined_goals

,team,home_goals,away_goals,total_goals
11,France,11,5.0,16.0
0,Argentina,11,4.0,15.0
10,England,10,3.0,13.0
20,Portugal,11,1.0,12.0
18,Netherlands,6,4.0,10.0
26,Spain,8,1.0,9.0
3,Brazil,7,1.0,8.0
7,Croatia,7,1.0,8.0
17,Morocco,1,5.0,6.0
12,Germany,1,5.0,6.0


Note: Since the world cup in 2022 took place in Qatar, they never played as the away team! This is why we used the **left** join in order to return Qatar's total home goals!